# 4 word2vec 속도 개선

## 4.1 word2vec 개선 1

앞 장의 CBOW모델은 단어 2개를 맥락으로 사용해, 이를 바탕으로 하나의 단어를 추측한다. 이때 입력 측 가중치$W_{in}$와의 행렬 곱으로 은닉층이 계산되고, 다시 출력층 가중치$W_{out}$와의 행렬 곱으로 각 단어의 점수를 구한다. 그리고 이 점수에 소프트맥스 함수를 적용해 각 단어의 출현 확률을 얻고, 이 확률을 정답 레이블과 비교하여 손실을 구한다.

말뭉치가 거대해 지면 다음과 같은 문제가 발생한다.

* 입력층의 원핫 표현과 가중치 행렬$W_{in}$의 곱 계산이 너무 느리다.
* 출력층의 가중치 행렬$W_{out}$과의 곱, 소프트맥스 계산도 느리다.

### 4.1.1 Embedding 계층

* Embedding 계층은 단어 ID에 해당하는 행을 추출하는 계층이다.
* Embedding 계층을 사용하면 단어의 분산 표현을 얻을 수 있다.

### 4.1.2 Embedding 계층 구현

In [1]:
import numpy as np

class Embedding:
  def __init__(self, W):
    self.params = [W]
    self.grads = [np.zeros_like(W)]
    self.idx = None
    
  def forward(self, idx):
    W = self.params
    self.idx = idx
    out = W[idx]
    return out
  
  def backward(self, dout):
    dW, = self.grads
    dW[...] = 0
    np.add.at(dW, self.idx, dout) # type: ignore
    return None

## 4.2 word2vec 개선 2

네거티브 샘플링(Negative Sampling)이란 기법을 사용하여 속도를 개선한다.

### 4.2.1 은닉층 이후 계산의 문제점

은닉층 이후 계산이 오래 걸리는 곳

* 은닉층의 뉴런과 가중치 행렬$W_{out}$의 곱
* 소프트맥스 함수의 계산

거대한 말뭉치를 다룰 때 이 행렬의 곱을 계산하려면 많은 계산량이 필요하다.

소프트맥스에서도 같은 문제가 발생한다.
$$
Y_{k} = \frac{exp(S_{k})}{\sum_{i=1}^{1,000,000}exp(S_{i})}
$$

k번째 단어의 점수$S_{k}$를 계산하려면 모든 단어의 점수를 계산해야 한다. 이 계산량은 말뭉치의 어휘 수에 비례한다.

### 4.2.2 다중 분류에서 이진 분류로

네거티브 샘플링 기법의 핵심 아이디어는 `이진 분류`이다. 이진 분류는 긍정(positive)과 부정(negative) 두 가지 중 하나를 판단하는 문제이다. 이를 이용하면 다중 분류 문제를 여러 개의 이진 분류 문제로 근사할 수 있다.

### 4.2.3 시그모이드 함수와 교차 엔트로피 오차

이진 분류 문제를 신경망으로 풀려면 점수에 `시그모이드 함수`를 적용해 확률로 변환하고, 손실을 구할 때는 손실 함수로 `교차 엔트로피 오차`를 사용한다.

$$
y = \frac{1}{1+exp(-x)}
$$

그래프는 S자 모양이며, 입력값$(x)$은 0에서 1 사이의 실수로 변환된다. 여기서 시그모이드 함수의 출력$y$는 확률로 해석할 수 있다.

손실은 `교차 엔트로피 오차`를 사용한다.

$$
L = -(tlogy + (1-t)log(1-y))
$$

$y$는 출력이고 $t$는 정답 레이블이다. 이 정답 레이블의 값은 0(부정) 또는 1(긍정)이다.

In [2]:
import numpy as np

class EmbeddingDot:
  """ Embedding 계층과 Dot 계층을 결합한 계층
  Parameters:
    embed: Embedding 계층
    params: 매개변수
    grads: 기울기
    cache: 순전파 시의 계산 결과를 잠시 유지하기 위한 변수
  """
  def __init__(self, W):
    """EmbeddingDot 계층의 초기화

    Args:
      W: 가중치
    
    Returns:
      None
    """
    self.embed = Embedding(W)
    self.params = self.embed.params
    self.grads = self.embed.grads
    self.cache = None
    
    
  def forward(self, h, idx):
    """순전파
    
    Args:
      h: 은닉층 뉴런
      idx: 단어 ID
      
    Returns:
      out: 내적 결과
    """
    target_W = self.embed.forward(idx)
    out = np.sum(target_W * h, axis=1)
    
    self.cache = (h, target_W)
    return out
  
  def backward(self, dout):
    """역전파
    
    Args:
      dout: 상류에서 넘어온 기울기
      
    Returns:
      dh: 하류로 흘려보낼 기울기
    """
    h, target_W = self.cache
    dout = dout.reshape(dout.shape[0], 1)
    
    dtarget_W = dout * h
    self.embed.backward(dtarget_W)
    dh = dout * target_W
    return dh

### 4.2.5 네거티브 샘플링

네거티브 샘플링은 긍정적인 예를 몇 개 선택하고, 그 외에는 모두 부정적인 예로 취급하는 방법이다.

긍정적인 예에 대해서는 시그모이드 함수의 출력을 1에 가깝게 만들고, 부정적인 예에 대해서는 0에 가깝게 만든다.

네거티브 샘플링 기법은 긍정적인 예를 타깃으로 한 경우의 손실과 부정적인 예를 몇 개 샘플링하여, 그 부정정직 예에 대한 손실을 더한 값을 **최종 손실**로 한다.

### 4.2.6 네거티브 샘플링의 샘플링 기법

네거티브 샘플링에서는 부정적인 예를 몇 개 샘플링하는데, 이 샘플링을 어떻게 할지가 중요하다.

말뭉치의 통계 데이터를 기초로 샘플링을 수행한다. 말뭉치에서 자주 등장하는 단어를 많이 추출하고, 드물게 등장하는 단어는 적게 추출한다.

말뭉치에서 단어별 출현 횟수를 바탕으로 확률분포를 구한다음, 그 확률분포에 따라서 샘플링을 수행한다. 확률분포대로 샘플링하므로 말뭉치에에서 자주 등장하는 단어는 선택될 가능성이 높고 드물게 등장하는 단어는 선택될 가능성이 낮다.

In [3]:
# 확률 분포에 따라 샘플링하기

import numpy as np

# 0 ~ 9까지의 숫자 중 하나를 무작위로 샘플링
print(np.random.choice(10))

2


word2vec의 네거티브 샘플링에서는 말뭉치에서 단어의 출현 횟수에 3/4제곱을 한 값을 확률분포로 사용한다.
$$
P'(w_{i}) = \frac{P(w_{i})^{0.75}}{\sum_{j}P(w_{j})^{0.75}} 
$$